In [2]:
import pandas as pd
import psycopg2
from psycopg2.extras import RealDictCursor
from datetime import timedelta

class PostgresDataHandler:
    def __init__(self, db_uri):
        self.conn = psycopg2.connect(db_uri, cursor_factory=RealDictCursor) # Connect to DB

    def get_data(self, tickers, date, lookback, freq="daily", columns = ['close']):
        """
        Function to easily query data from postgres database

        Args:
            db_uri: Valid connection Object

        Returns:
            
        """
        end_date = pd.to_datetime(date) # Convert to pandas datetime object
        if freq != "daily":
            raise ValueError("Only daily data is allowed for now")

        start_date = end_date - timedelta(days=lookback) # Calculate start date

        selected_cols = ', '.join(['date', 'ticker'] + columns)

        sql = f"""
SELECT {selected_cols}
FROM ohlcv
WHERE ticker = ANY(%s)
  AND date BETWEEN %s AND %s
ORDER BY date
        """

        with self.conn.cursor() as cur:
            cur.execute(sql, (tickers, start_date, end_date))
            rows = cur.fetchall()


        df = pd.DataFrame(rows)
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df = df.pivot(index='timestamp', columns='ticker', values='close')
        return df


    def close(self):
        self.conn.close()


In [ ]:
db_uri = f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASS')}@" \
         f"{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
